In [ ]:
import pandas as pd
import numpy as np


#  Olympic data set analysis

Often people wonder if how a country performs is related to its, Economic status. it isbelived that people from more economically adept places have a better chance at plaacing at the games. people also wonder if hosting an olypmic games provides a siginficant boost to the economy that they are in. This anlysis will inclue the ETL, of various relevant documents such as medal data, olympic perfromance and partipatyion data, into an organisaed warehouse where visualisations will stem from to answer the questions above.

0lypmic performance will be measures in medal counts of that partcular country, and economic perforamce indicated by the gross domestic product of that partcular country (GDP), the data used will stem from 2000 olympics to the current 2023 olympic games to provide a recent but comprhensive suite of data.

## ETL Process 
Below we conduct the etl, process creating our different dimensions and then fact tables. once we do that we will then connect to a pgadmin remost server and from there build hierachies and so forth.